## Ngram Model (updated)

This is the version which fix the bugs in the model.

## input the sent-lable pickle data

In [1]:
import pickle 
with open('/Users/apple/Documents/GitHub/Argument-Scoring-System/pretrained_model/labeled_essay_dics.pickle', 'rb') as handle:
    label_sents = pickle.load(handle)

In [2]:
# compute and store and the {sents: label} into one dic
all_label_sents = {}
for key in label_sents.keys():
    this_essay = label_sents[key]
    for s in this_essay.keys():
        all_label_sents[s] = this_essay[s]

## data cleaning 

In [3]:
import re
import itertools
from collections import Counter

In [4]:
"""
Original taken from https://github.com/dennybritz/cnn-text-classification-tf
"""


def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [5]:
def clear_split_str(s):
    strip_s = s.strip()
    clear_s = clean_str(strip_s)
    s_text = clear_s.split(" ")
    return s_text

In [6]:
# clear data and store in one dictionary
X = []
y = []
for key in all_label_sents.keys():
    this_value = all_label_sents[key]
    clear_key = clear_split_str(key)
    X.append(clear_key)
    y.append(this_value)

In [7]:
print (y[1:10])

['MajorClaim', 'Premise', 'Premise', 'Empty', 'Empty', 'Empty', 'Premise', 'Premise', 'Premise']


## remove low frequency words

In [8]:
from nltk import FreqDist

In [9]:
with open('argument_words.pickle', 'rb') as handle:
    common_words = pickle.load(handle)

In [10]:
def freq_words(number, words):
    '''
    return the words which the frequency is higher than the given number
    number: the lowest value of frequency
    words: the cleared words list
    '''
    word_freq = FreqDist(words).most_common(1000)
    words_list = [key for (key, value) in word_freq if value > number]
    return words_list

In [11]:
input_words = common_words['all']
# word_list = freq_words(28, input_words)
word_list = freq_words(5, input_words)

In [12]:
new_X = []
new_y = y

In [13]:
for item in X:
    new_sent = []
    for w in item:
        if w in word_list:
            new_sent.append(w)
    new_X.append(new_sent)

***error analyze***:
* after removing the low frequency, the result can improve about 5%

## remove low frequency words

In [14]:
from sklearn.cross_validation import train_test_split

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(
#                                                 new_X, 
                                                   X, 
                                                   y,
                                                   test_size = 0.1,
#                                                    random_state=30
                                                   )

## sepratate training data by class

In [16]:
claim_list = []
premise_list = []
major_claim_list = []
other_list = []

for i in range(0, len(X_train)):
    this_value = y_train[i]
    clear_key = X_train[i]
    if this_value == "Premise":
        premise_list.append(clear_key)
    elif this_value == "Claim":
        claim_list.append(clear_key)
    elif this_value == "MajorClaim":
        major_claim_list.append(clear_key)
    else:
        other_list.append(clear_key)

In [17]:
# print (claim_list[1:10])
print (len(premise_list) + len(claim_list) + len(major_claim_list) + len(other_list) + len(y_test))
# print ()

7077


## ngram model 

In [18]:
import collections
from collections import defaultdict

In [19]:
def ngram1_4(sent_list):
    '''
    computer the 1 to 4 gram, and return the list
    sent_list: list of sents
    return: a dic includes one to four grams 
    '''
    # unigram
    uni_num = defaultdict(float)
    total_uni_num = 0
    for sent in sent_list:
        for i in range(len(sent)-1+1):
            this_key = ' '.join(sent[i:i+1])
            uni_num[this_key] = uni_num.get(this_key,0) + 1
            total_uni_num = total_uni_num + 1
    word_pro = {}
    for key in uni_num:
        word_pro[key] = uni_num[key]/float(total_uni_num)
    
    
    #bigram 
    bi_num = defaultdict(float)
    for sent in sent_list:
        for i in range(len(sent)-2+1):
            this_key = ' '.join(sent[i:i+2])
            bi_num[this_key] = bi_num.get(this_key,0) + 1
    bi_pro = {}        
    for key in bi_num:
        pre_key = key.split()[0]
        bi_pro[key] = bi_num[key]/float(uni_num[pre_key])
        
    # trigram
    tri_num = defaultdict(float)
    for sent in sent_list:
        for i in range(len(sent)-3+1):
            this_key = ' '.join(sent[i:i+3])
            tri_num[this_key] = tri_num.get(this_key,0) + 1
    tri_pro = {}
    for key in tri_num:
        pre_key = ' '.join(key.split()[0:2])
        tri_pro[key] = tri_num[key]/float(bi_num[pre_key])
        
    # fourgram
    four_num = defaultdict(float)
    for sent in sent_list:
        for i in range(len(sent)-4+1):
            this_key = ' '.join(sent[i:i+4])
            four_num[this_key] = four_num.get(this_key,0) + 1
    four_pro = {}
    for key in four_num:
        pre_key = ' '.join(key.split()[0:3])
        four_pro[key] = four_num[key]/float(tri_num[pre_key])
    
    gram_dic = {}
    gram_dic[1] = word_pro
    gram_dic[2] = bi_pro
    gram_dic[3] = tri_pro
    gram_dic[4] = four_pro
    return gram_dic

In [20]:
# ngram1_4([['I', 'am', 'crazy', 'am','good'],['I','bad','I'],['I','am','not','crazy']])

In [21]:
# the 1 - 4 grams for every class
claim_grams = ngram1_4(claim_list)
premise_grams = ngram1_4(premise_list)
major_claim_grams = ngram1_4(major_claim_list)
other_grams = ngram1_4(other_list)

## build classifier

In [22]:
from math import log

In [23]:
def get_grams(sent, n):
    gram_list = []
    for i in range(len(sent)-n+1):
        gram_list.append(' '.join(sent[i:i+n]))
    return gram_list

In [24]:
import sys
epslon = sys.float_info.epsilon

In [25]:
def sent_prob(keys, pro_dic):
    result = 0.0
    for key in keys:
#         result = result + (log(pro_dic.get(key, epslon)))
        result = result + (log(pro_dic.get(key, epslon)))
    return result

In [26]:
log(1)
# claim_grams 
# premise_grams 
# major_claim_grams
# other_grams 

0.0

In [27]:
def sent_pro_dic(sent):
    claim_sum = 0.0
    premise_sum = 0.0
    major_claim_sum = 0.0
    others_sum = 0.0
    
    p = {}
    p[1] = 0.1
    p[2] = 0.2
    p[3] = 0.3
    p[4] = 0.4
    
    for i in range(1,5):
        this_key = get_grams(sent, i)
        claim_sum = claim_sum + p[i] * sent_prob(this_key, claim_grams[i])
        premise_sum = premise_sum + p[i] * sent_prob(this_key, premise_grams[i])
        major_claim_sum = major_claim_sum + p[i] * sent_prob(this_key, major_claim_grams[i])
        others_sum = others_sum + p[i] * sent_prob(this_key, other_grams[i])
    
    pro_dic = {}
    pro_dic['Claim'] = claim_sum
    pro_dic['Premise'] = premise_sum
    pro_dic['MajorClaim'] = major_claim_sum
    pro_dic['Empty'] = others_sum
    return pro_dic

In [28]:
def max_class(values):
    key = max(values, key=lambda i: values[i])
    return key

## reverse ngram

In [29]:
# just reverse the sents, and compute with the 1-4gram model to compute the result

In [30]:
def reverse_ngram(sent_list):
    new_sent_list = []
    for sent in sent_list:
        new_sent_list.append(list(reversed(sent)))
    
    # compute the ngram 
    gram_dic = ngram1_4(new_sent_list)
    
    return gram_dic
    

In [31]:
# the 1 - 4 reverse grams for every class
rev_claim_grams = reverse_ngram(claim_list)
rev_premise_grams = reverse_ngram(premise_list)
rev_major_claim_grams = reverse_ngram(major_claim_list)
rev_other_grams = reverse_ngram(other_list)

In [32]:
def sent_pro_dic_all(sent):
    '''
    compute the forward and backward order sent prob.
    '''
    rev_sent = list(reversed(sent))
    rev_claim_sum = 0.0
    rev_premise_sum = 0.0
    rev_major_claim_sum = 0.0
    rev_others_sum = 0.0
    
    p = {}
    p[1] = 0.1
    p[2] = 0.2
    p[3] = 0.3
    p[4] = 0.4
    
    for i in range(1,5):
        this_key = get_grams(rev_sent, i)
        rev_claim_sum = rev_claim_sum + p[i] * sent_prob(this_key, rev_claim_grams[i])
        rev_premise_sum = rev_premise_sum + p[i] * sent_prob(this_key, rev_premise_grams[i])
        rev_major_claim_sum = rev_major_claim_sum + p[i] * sent_prob(this_key, rev_major_claim_grams[i])
        rev_others_sum = rev_others_sum + p[i] * sent_prob(this_key, rev_other_grams[i])
    
    pro_dic = sent_pro_dic(sent)
    pro_dic['Claim'] = pro_dic['Claim'] + rev_claim_sum
    pro_dic['Premise'] = pro_dic['Premise'] + rev_premise_sum
    pro_dic['MajorClaim'] = pro_dic['MajorClaim'] + rev_major_claim_sum
    pro_dic['Empty'] = pro_dic['Empty'] + rev_others_sum
    
    return pro_dic
    

## predict and error analyze

In [33]:
from sklearn.metrics import accuracy_score, f1_score

In [34]:
y_pred = []
for item in X_test:
#     this_value = max_class(sent_pro_dic(item))
    this_value = max_class(sent_pro_dic_all(item))
    y_pred.append(this_value)

In [35]:
accuracy = accuracy_score(y_pred,y_test)
print (accuracy)

0.539548022599


## Error analyze 
* The improvement is not so obvious
* Reducing the low frequency words improve the pro by 5%
* Computing the reverse ngram improve the result by 0.5%
* Need to recheck the code to find the bug there

In [36]:
for i in range(0, len(X_test)):
    this_value = max_class(sent_pro_dic(X_test[i]))
    if this_value != y_test[i]:
        print (X_test[i])
        print (this_value, y_test[i])
        print ()

['to', 'start', 'with', ',', 'since', 'technology', 'was', 'introduced', ',', 'student', 'have', 'been', 'exposed', 'to', 'a', 'wide', 'range', 'of', 'information', ',', 'which', 'neccessarily', 'broaden', 'their', 'minds']
Premise Claim

['first', ',', 'in', 'my', 'view', 'spending', 'too', 'much', 'money', 'for', 'equipping', 'the', 'gyms', 'or', 'constructing', 'football', 'basketball', 'courts', 'are', 'really', 'not', 'necessary', 'to', 'encourage', 'students', 'to', 'do', 'sports']
Premise Claim

['as', 'much', 'as', 'their', 'events', 'might', 'were', 'bitter', ',', 'in', 'their', 'adulthood', 'they', 'are', 'sweet']
Premise Empty

['everyday', ',', 'companies', 'offer', 'a', 'huge', 'number', 'of', 'different', 'products', 'to', 'consumers']
Premise Empty

['third', ',', 'there', 'are', 'many', 'ways', 'to', 'fund', 'a', 'college', 'education']
MajorClaim Claim

['they', 'start', 'to', 'pay', 'attention', 'to', 'mental', 'requirement', 'like', 'happiness']
Premise Empty

['all'

* the essays labeled by different people maybe different. 
* the 